In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

# Load WikiText-103 dataset
# dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

dataset = load_from_disk("../processed_datadir/wikitext-103-raw-train")

In [6]:
print(dataset.features)

{'text': Value(dtype='string', id=None)}


In [2]:
print(len(dataset))

1801350


In [2]:
print(dataset[1])

{'text': ' = Valkyria Chronicles III = \n'}


In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Features
story_data = {
    "text":[]
}
index = 1
while True:
    if dataset[index]["text"].count("=") == 2:
        story_data["text"].append(dataset[index]["text"])
    else:
        story_data["text"][-1] = story_data["text"][-1] + dataset[index]["text"]
    index += 1
    if index == len(dataset):
        break
    if index % 10000 == 1:
        print(index)
        


In [ ]:
for i in range(10):
    print(story_data["text"][i])

In [8]:
print(len(story_data["text"]))

29525


In [7]:


# 创建数据集
dataset = Dataset.from_dict(story_data)

# 保存数据集
dataset.save_to_disk("../processed_datadir/wikitext-103-story-train")

Saving the dataset (0/2 shards):   0%|          | 0/29525 [00:00<?, ? examples/s]

In [ ]:
# read next fast speed
# dataset.save_to_disk("wikitext-103-raw-train")

In [ ]:
# for i in range(0, 1000):
#     print(dataset[i])

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../tokenizer_save/tokenizer-gpt2-512/")

In [9]:
import multiprocessing

num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")
# 害怕太多pad，导致模型只预测pad，所以只取最大长度的文本,后来一想这是不太可能的，attention会直接遮蔽，但是这样其实没有预测结束符
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True,   # 在这就不带文本了
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True,return_length=True)
    input_ids_list = []
    for length, input_ids in zip(tokenized_inputs["length"], tokenized_inputs["input_ids"]):
        if length == tokenizer.model_max_length:
            input_ids_list.append(input_ids)
        
    return {"input_ids": input_ids_list}

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)



The max length for the tokenizer is: 512


Map (num_proc=16):   0%|          | 0/29525 [00:00<?, ? examples/s]

In [11]:
# Split preprocessed dataset into train, validation, and test sets
splits = tokenized_datasets.train_test_split(test_size=0.05)
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_from_disk("../processed_datadir/wikitext-103-story-test/")
})

### Then we can construct our own wikitext test data
The biggest difference is that train and evaluate always stay with CrossEntry Loss, and won't care the real text effect, but text should have the overflowing tokens for GPT2
 

In [13]:
def group_texts(examples):
    test_length = 512
    tokenized_inputs = tokenizer(examples["text"], truncation=True,   # 在这就不带文本了
        max_length=test_length, return_overflowing_tokens=True,return_length=True)
    input_ids_list = []
    prediction_labels = []
    for length, input_ids in zip(tokenized_inputs["length"], tokenized_inputs["input_ids"]):
        if length == test_length:
            input_ids_list.append(input_ids[:test_length//2])
            prediction_labels.append(input_ids[test_length//2:])
        
    return {"input_ids": input_ids_list,"prediction_labels": prediction_labels}

preprocessed_splits["test"] = preprocessed_splits["test"].map(group_texts,batched=True, remove_columns=["text"],num_proc=4)

Map (num_proc=4):   0%|          | 0/62 [00:00<?, ? examples/s]

In [14]:
print(len(preprocessed_splits["test"]))

524


In [ ]:
for i in range(10):
    print(preprocessed_splits["test"][i])

In [9]:
try:
    preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns(["overflowing_tokens_ids"])
except:
    pass
try:
    preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns(["num_truncated_tokens"])
except:
    pass

In [16]:
preprocessed_splits.save_to_disk("../processed_datadir/wikitext-103-gpt2-story-train512-test256")

Flattening the indices:   0%|          | 0/204951 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/204951 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10787 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10787 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/524 [00:00<?, ? examples/s]